In [ ]:
# with open("/kaggle/input/Alice-adventure/allice.txt","r") as f:
#     text = f.read().replace("\n\n"," ").replace("\n"," ").replace("*","   ")
#     chapters = text.split("CHAPTER")[1:]

# chapter1 = chapters[0][2:][1:] # skipping first character of the chapter

# nlp = spacy.load("en_core_web_sm")
# doc = nlp(chapter1)
# sentences = list(doc.sents)
# print(sentences[0])
# # print(chapters[0])

# LLaMA

https://github.com/facebookresearch/llama

In [ ]:
# 
# Tokenizer
from sentencepiece import SentencePieceProcessor # This has encode-decode
class Tokenizer:
    def __init__(self, model_path: str):
        1. reload tokenizer # self.sp_model = SentencePieceProcessor(model_file=model_path)
        2. BOS / EOS token IDs
    def encode(self, s: str, bos: bool, eos: bool) -> List[int]:
        1. encode model & add to bos_id,eos_id 
    def decode(self, t: List[int]) -> str:
        1. decode model

# Generation
from llama.tokenizer import Tokenizer
from llama.model import Transformer

class LLaMA:
    def __init__(self, model: Transformer, tokenizer: Tokenizer):
    def sample_top_p(probs, p):
        1. probs_sort, probs_idx by sorting probs in descending ,dim=-1
        2. probs_sum & mask = checker probs_sum - probs_sort > p , probs_sort[mask]=0.0
        3. probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
        4. next_token = torch.multinomial(probs_sort, num_samples=1)
           next_token = torch.gather(probs_idx, -1, next_token)
    
    def generate(self,prompts: List[str],max_gen_len: int,temperature: float = 0.8,top_p: float = 0.95,) -> List[str]:
        
        1. bcz=prompts size <= params.max_batch_size size
        2. prompt_tokens = [tokenize each prompts]
        3. get min_prompt_size & max_prompt_size for each prompts : List[str]
        4. get total_len = min(params.max_seq_len, max_gen_len + max_prompt_size)
        5. tokens = torch.full((bsz, total_len), self.tokenizer.pad_id).cuda().long()
            tokens[k, : len(t)] = torch.tensor(t).long() for each prompt_tokens
            input_text_mask = tokens not having tokenized padding
            assign start_pos , prev_pos
        6. for cur_pos in range(start_pos,total_len):
                logits = forward pass with (tokens[:, prev_pos:cur_pos], prev_pos)
                if temperature > 0:
                    probs = use softmax with logits & temperature, get dim=-1
                    next_token = sample_top_p(probs, top_p).reshape(-1)
                else:
                    next_token = argmax(logits,dim=-1).reshape(-1)
            
                only replace tokens[:, cur_pos] if prompt has already been generated -> input_text_mask[:, cur_pos] : bool 
                tokens[:, cur_pos] = next_token
                prev_pos = cur_pos
        7. get each token for tokens.tolist() & cut to max_gen_len or eos_id if any then decode

In [ ]:
# model
import fairscale.nn.model_parallel.initialize as fs_init
from fairscale.nn.model_parallel.layers import (
    ParallelEmbedding,
    RowParallelLinear,
    ColumnParallelLinear,
)

@dataclass
class ModelArgs:
    ...
    vocab_size: int = -1  # defined later by tokenizer
    multiple_of: int = 256  # make SwiGLU hidden layer size multiple of large power of 2
        
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__(),assign eps & self.weight = nn.Parameter(torch.ones(dim))
    def _norm(self, x): return x* rsqrt(...)
    def forward(self, x): return output * weight # output = _norm(x)

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    1. pre-computation for rotary embedding , use torch.polar() to get dtype=complex64 

def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    # here 0 <= 1 < ndim=x.ndim , freqs_cis must have same dimension as x except for 1 & ndim-1
    return freqs_cis.view(*shape)

def apply_rotary_emb(xq: torch.Tensor,xk: torch.Tensor,) -> Tuple[torch.Tensor, torch.Tensor]:
    1. find xq_ & xk_ by using proper field from xq & xk to get real ,img components
    2. freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    3. xq_out,xk_out = real components but with rotatry embeddings

class Attention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_local_heads = args.n_heads // fs_init.get_model_parallel_world_size()
        self.head_dim = args.dim // args.n_heads
        1. use ColumnParallelLinear toget wq,wk,wv,wo; set cache_k,cache_v

    def forward(self, x: torch.Tensor, start_pos: int, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor]):
        bsz, seqlen, _ = x.shape,xq, xk, xv 
        1. xq, xk = apply_rotary_emb(xq, xk, freqs_cis=freqs_cis) & put into cache_k(xq),cache_c(xq)
        2. get keys,values,scores -> add score + mask -> Softmax => score
        3. return wo(score @ values)

class FeedForward(nn.Module):
    def __init__(self,dim: int,hidden_dim: int,multiple_of: int,):
        super().__init__(),hidden_dim,assign w1,w2,w3 using ColumnParallelLinear

    def forward(self, x): return sigmoid linear unit applied on w1,w2,w3

    
class TransformerBlock(nn.Module):
    def __init__(self, layer_id: int, args: ModelArgs):
        super().__init__(),get attention,feed-forward,ffn_norm & other head,dim parameters

    def forward(self, x: torch.Tensor, start_pos: int, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor]):
        # both are for forward pass
        1. calculate h = x + attention(attention_norm(x),start_pos,freqs_cis, mask)
        2. return out = h + feed_forward(ffn_norm(h))


class Transformer(nn.Module):
    def __init__(self, params: ModelArgs):
        super().__init__(),assign params,vocab_size,tok_embed,layers,norm ->freqs_cis,output
        
    @torch.inference_mode()
    def forward(self, tokens: torch.Tensor, start_pos: int):
        _bsz, seqlen = tokens.shape
        1. h = tok_embeddings(token) -> freqs_cis(h)
        2. seqlen > 1: add mask() not the complete but triu part
        3. h = layer(h, start_pos, freqs_cis, mask) for each layers -> norm(h)
        4. output = compute with last logits of output(h[:, -1, :])
        

# HuggingFace - Transformers

all models ->  https://github.com/huggingface/transformers/tree/main/src/transformers/models

BERT (https://github.com/huggingface/transformers/tree/main/src/transformers/models/bert)

        1. model - https://github.com/huggingface/transformers/blob/v4.29.1/src/transformers/models/bert/modeling_bert.py
        2. tokenizer - https://github.com/huggingface/transformers/blob/v4.29.1/src/transformers/models/bert/tokenization_bert_fast.py

GPT-2 (https://github.com/huggingface/transformers/tree/main/src/transformers/models/gpt2)

Alpaca (https://github.com/tatsu-lab/stanford_alpaca)

LoRA (https://github.com/microsoft/LoRA)

GPT4ALL (https://github.com/nomic-ai/gpt4all) -> advanced not needed

### Transformer for Long Sentences -> LongFormer

LongFormer (https://github.com/huggingface/transformers/tree/main/src/transformers/models/longformer)

# HuggingFace - Tokenizers

Pre-Tokenizer : Breaks sentence into word to avoid overlap then we can use Tokenizer

Tokenizer Pipeline
-----------------------

    normalization
    pre-tokenization
    model
            models.BPE
            models.Unigram
            models.WordLevel
            models.WordPiece
    post-processing
    
 BPE (https://github.com/soaxelbrooke/python-bpe)
 

# HuggingFace - Diffusers 

all diffusers -> https://github.com/huggingface/diffusers/tree/main/src/diffusers/models

In [ ]:
#

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
epochs = 10

In [3]:
train_dataset = datasets.MNIST(root='data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = datasets.MNIST(root='data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 94631985.73it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 71047327.76it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 26605517.37it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5719162.04it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# NN

In [4]:
input_size = 784 #[28X28]
hidden_layers = 50


In [5]:
class NN(nn.Module):
    def __init__(self,input_size,hidden_layers,num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_layers)
        self.fc2 = nn.Linear(hidden_layers,num_classes)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# model = NN(784,50,10)
# x = torch.rand(64,784).to(device)
# model(x).shape

In [6]:
model = NN(input_size,hidden_layers,num_classes).to(device)

optimizer = optim.SGD(params=model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,factor=0.01)

In [7]:
for epoch in range(epochs):
    losses = []
    for batch_idx,(data,target) in tqdm(enumerate(train_loader)):
        data = data.to(device)
        target = target.to(device)
        data = data.reshape(data.shape[0],-1) # flatten : (64,1X768)
        score = model(data) # [64,10]
        loss = criterion(score,target)
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"Cost for epoch={epoch} is {mean_loss:.2f}")

938it [00:10, 91.50it/s] 


Cost for epoch=0 is 2.25


938it [00:08, 112.19it/s]


Cost for epoch=1 is 2.09


938it [00:07, 118.97it/s]


...

Cost for epoch=7 is 0.80


938it [00:07, 118.37it/s]


Cost for epoch=8 is 0.72


938it [00:08, 114.12it/s]

Cost for epoch=9 is 0.66


In [9]:

def check_accuracy(model,loader):
    if loader.dataset.train:
        print("Training Accuracy")
    else:
        print("Testing Accuracy")
    samples = 0
    corrects = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to(device)
            y=y.to(device)
            x = x.reshape(x.shape[0],-1)
            score = model(x) # [64,10]
            _,pred_index = score.max(1) #[64] ,don't care about the value
            corrects += (pred_index==y).sum()
            samples += pred_index.size(0)
        print(f"Correct = {corrects}/{samples} Accuracy = {float(corrects)/float(samples)*100:.2f}")
    model.train()
    
    
check_accuracy(model,train_loader)
check_accuracy(model,test_loader)

Training Accuracy


100%|██████████| 938/938 [00:07<00:00, 124.92it/s]


Correct = 51065/60000 Accuracy = 85.11
Testing Accuracy


100%|██████████| 157/157 [00:01<00:00, 115.79it/s]

Correct = 8607/10000 Accuracy = 86.07


# CNN

In [ ]:
n_channels = 1

In [ ]:
class CNN(nn.Module):
    def __init__(self,n_channels,num_classes):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=n_channels,out_channels=8,kernel_size=(3,3),
                              padding=(1,1),stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),
                              padding=(1,1),stride=(1,1))
        self.fc = nn.Linear(16*7*7,num_classes)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc(x)
        return x
    
# model = CNN(1,10)
# x = torch.rand(64,1,28,28).to(device)
# model(x).shape  # [64,10]

In [ ]:
model = CNN(n_channels,num_classes).to(device)

optimizer = optim.SGD(params=model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,factor=0.01)

In [ ]:
for epoch in range(epochs):
    losses = []
    for batch_idx,(data,target) in tqdm(enumerate(train_loader)):
        data = data.to(device)
        target = target.to(device)
        #data = data.reshape(data.shape[0],-1) # flatten : (64,1X768)
        score = model(data) # [64,10]
        loss = criterion(score,target)
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"Cost for epoch={epoch} is {mean_loss:.2f}")
    

In [ ]:
        
def check_accuracy(model,loader):
    if loader.dataset.train:
        print("Training Accuracy")
    else:
        print("Testing Accuracy")
    samples = 0
    corrects = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to(device)
            y=y.to(device)
            #x = x.reshape(x.shape[0],-1)
            score = model(x) # [64,10]
            _,pred_index = score.max(1) #[64] , don't care about the value
            corrects += (pred_index==y).sum()
            samples += pred_index.size(0)
        print(f"Correct = {corrects}/{samples} Accuracy = {float(corrects)/float(samples)*100:.2f}")
    model.train()
    
    
    
check_accuracy(model,train_loader)
check_accuracy(model,test_loader)

# RNN

In [10]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_layers = 256

In [11]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_layers,num_layers,num_classes):
        super(RNN,self).__init__()
        self.hidden_layers = hidden_layers
        self.num_layers = num_layers
        # [NXtime_sequenceXfeatures] -> batch_first=True
        self.rnn = nn.RNN(input_size,hidden_layers,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_layers*sequence_length,num_classes)
        
    def forward(self,x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_layers).to(device)
        out,_= self.rnn(x,h0) # don't need to store hidden_state
        out = out.reshape(out.shape[0],-1) #[64,256*sequence_length]
        out = self.fc(out)
        return out
    
# model = RNN(28,256,2,10)
# x = torch.rand(64,28,28).to(device)
# model(x).shape

In [12]:
model = RNN(input_size,hidden_layers,num_layers,num_classes).to(device)

optimizer = optim.SGD(params=model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,factor=0.01)

In [13]:
for epoch in range(epochs):
    losses = []
    for batch_idx,(data,target) in tqdm(enumerate(train_loader)):
        data = data.to(device).squeeze(1) # [64,1,28,28] -> [64,1*28,28]
        target = target.to(device)
        #data = data.reshape(data.shape[0],-1) # flatten : (64,1X768)
        score = model(data) # [64,10]
        loss = criterion(score,target)
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"Cost for epoch={epoch} is {mean_loss:.2f}")

938it [00:11, 84.00it/s]


Cost for epoch=0 is 2.10


938it [00:10, 85.55it/s]


Cost for epoch=1 is 1.09


938it [00:10, 89.17it/s]


...

Cost for epoch=7 is 0.37


938it [00:10, 88.17it/s]


Cost for epoch=8 is 0.35


938it [00:10, 88.52it/s]

Cost for epoch=9 is 0.34


In [14]:

def check_accuracy(model,loader):
    if loader.dataset.train:
        print("Training Accuracy")
    else:
        print("Testing Accuracy")
    samples = 0
    corrects = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to(device).squeeze(1)
            y=y.to(device)
            #x = x.reshape(x.shape[0],-1)
            score = model(x) # [64,10]
            _,pred_index = score.max(1) #[64] , don't care about the value
            corrects += (pred_index==y).sum()
            samples += pred_index.size(0)
        print(f"Correct = {corrects}/{samples} Accuracy = {float(corrects)/float(samples)*100:.2f}")
    model.train()
    
    
    
check_accuracy(model,train_loader)
check_accuracy(model,test_loader)

Training Accuracy


100%|██████████| 938/938 [00:08<00:00, 114.72it/s]


Correct = 54203/60000 Accuracy = 90.34
Testing Accuracy


100%|██████████| 157/157 [00:01<00:00, 115.73it/s]

Correct = 9053/10000 Accuracy = 90.53


# LSTM

In [66]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layers, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_layers = hidden_layers
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layers, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_layers, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_layers).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_layers).to(device)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        out, _ = self.lstm( x, (h0, c0))  
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out) # no need to reshape if we're taking only last layer
        return out

In [67]:
model = LSTM(input_size,hidden_layers,num_layers,num_classes).to(device)

optimizer = optim.SGD(params=model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,factor=0.01)

In [68]:
for epoch in range(epochs):
    losses = []
    for batch_idx,(data,target) in tqdm(enumerate(train_loader)):
        data = data.to(device).squeeze(1) # [64,1,28,28] -> [64,1*28,28]
        target = target.to(device)
        #data = data.reshape(data.shape[0],-1) # flatten : (64,1X768)
        score = model(data) # [64,10]
        loss = criterion(score,target)
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"Cost for epoch={epoch} is {mean_loss:.2f}")

0it [00:00, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x7168 and 256x10)

In [18]:
def check_accuracy(model,loader):
    if loader.dataset.train:
        print("Training Accuracy")
    else:
        print("Testing Accuracy")
    samples = 0
    corrects = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to(device).squeeze(1)
            y=y.to(device)
            #x = x.reshape(x.shape[0],-1)
            score = model(x) # [64,10]
            _,pred_index = score.max(1) #[64] , don't care about the value
            corrects += (pred_index==y).sum()
            samples += pred_index.size(0)
        print(f"Correct = {corrects}/{samples} Accuracy/epoch = {float(corrects)/float(samples)*100:.2f}")
    model.train()
    
    
    
check_accuracy(model,train_loader)
check_accuracy(model,test_loader)

Training Accuracy


100%|██████████| 938/938 [00:08<00:00, 113.01it/s]


Correct = 6742/60000 Accuracy/epoch = 11.24
Testing Accuracy


100%|██████████| 157/157 [00:01<00:00, 110.95it/s]

Correct = 1135/10000 Accuracy/epoch = 11.35


# Bidirectional LSTM

In [30]:
class BLSTM(nn.Module):
    def __init__(self, input_size, hidden_layers, num_layers, num_classes):
        super(BLSTM, self).__init__()
        self.hidden_layers = hidden_layers
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layers, num_layers, batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_layers * 2, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_layers).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_layers).to(device)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        out, _ = self.lstm(x, (h0, c0))  
        #out = out.reshape(out.shape[0], -1)
        out = self.fc(out[:,-1,:]) 
        return out

In [31]:
model = BLSTM(input_size,hidden_layers,num_layers,num_classes).to(device)

optimizer = optim.SGD(params=model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,factor=0.01)

In [33]:
for epoch in range(epochs):
    losses = []
    for batch_idx,(data,target) in tqdm(enumerate(train_loader)):
        data = data.to(device).squeeze(1) # [64,1,28,28] -> [64,1*28,28]
        target = target.to(device)
        #data = data.reshape(data.shape[0],-1) # flatten : (64,1X768)
        score = model(data) # [64,10]
        loss = criterion(score,target)
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"Cost for epoch={epoch} is {mean_loss:.2f}")

0it [00:00, ?it/s]


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
def check_accuracy(model,loader):
    if loader.dataset.train:
        print("Training Accuracy")
    else:
        print("Testing Accuracy")
    samples = 0
    corrects = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to(device).squeeze(1)
            y=y.to(device)
            #x = x.reshape(x.shape[0],-1)
            score = model(x) # [64,10]
            _,pred_index = score.max(1) #[64] , don't care about the value
            corrects += (pred_index==y).sum()
            samples += pred_index.size(0)
        print(f"Correct = {corrects}/{samples} Accuracy/epoch = {float(corrects)/float(samples)*100:.2f}")
    model.train()
    
    
    
check_accuracy(model,train_loader)
check_accuracy(model,test_loader)

In [ ]:
#